In [1]:
import pandas as pd
import numpy as np
import os
import itertools
pd.set_option("display.max_columns", 999)

In [2]:
data_path = os.path.join("..", "data", "campeonato-brasileiro-full.csv")
data = pd.read_csv(data_path, parse_dates=["Data"])
data["Clube 1"] = data["Clube 1"].str.upper()
data["Clube 2"] = data["Clube 2"].str.upper()
data["Arena"] = data["Arena"].str.upper()
data.columns = [x.replace(" ", "_") for x in data.columns]
data.head()

,Horário,Dia,Data,Clube_1,Clube_2,Vencedor,Rodada,Arena,Clube_1_Gols,Clube_2_Gols,Clube_1_Estado,Clube_2_Estado,Estado_Clube_Vencedor
0,16h00,Sábado,2000-07-29,FLUMINENSE,BAHIA,Fluminense,1ª Rodada,MARACANÃ,2,0,RJ,BA,RJ
1,16h00,Sábado,2000-07-29,VASCO,SPORT,Sport,1ª Rodada,SÃO JANUÁRIO,0,2,RJ,PE,PE
2,16h00,Sábado,2000-07-29,VITÓRIA,PALMEIRAS,Vitória,1ª Rodada,BARRADÃO,4,1,ES,SP,ES
3,17h00,Domingo,2000-07-30,BOTAFOGO-RJ,ATLÉTICO-MG,-,1ª Rodada,CAIO MARTINS,0,0,RJ,MG,Empate
4,18h30,Domingo,2000-07-30,JUVENTUDE,FLAMENGO,-,1ª Rodada,ALFREDO JACONI,1,1,RS,RJ,Empate


In [3]:
data = data.drop(columns=["Horário", "Estado_Clube_Vencedor", "Rodada", "Dia", "Vencedor", "Arena"])
data = data.query("Data>='2006'")

In [4]:
data.head()

,Data,Clube_1,Clube_2,Clube_1_Gols,Clube_2_Gols,Clube_1_Estado,Clube_2_Estado
2619,2006-04-15,VASCO,INTERNACIONAL,1,1,RJ,RS
2620,2006-04-15,JUVENTUDE,PARANÁ,1,0,RS,PR
2621,2006-04-16,SANTA CRUZ,FIGUEIRENSE,0,0,PE,SC
2622,2006-04-16,GOIÁS,SANTOS,0,0,GO,SP
2623,2006-04-16,BOTAFOGO-RJ,FORTALEZA,1,0,RJ,CE


In [5]:
def return_match_result(row):
    home_score = row["Clube_1_Gols"]
    away_score = row["Clube_2_Gols"]
    if home_score > away_score:
        return "Mandante"
    elif away_score > home_score:
        return "Visitante"
    else:
        return "Empate"
    
data["Vencedor_Cat"] = data.apply(return_match_result, axis=1)
data = data.sort_values("Data", ascending=True)
data.head()

,Data,Clube_1,Clube_2,Clube_1_Gols,Clube_2_Gols,Clube_1_Estado,Clube_2_Estado,Vencedor_Cat
2619,2006-04-15,VASCO,INTERNACIONAL,1,1,RJ,RS,Empate
2620,2006-04-15,JUVENTUDE,PARANÁ,1,0,RS,PR,Mandante
2621,2006-04-16,SANTA CRUZ,FIGUEIRENSE,0,0,PE,SC,Empate
2622,2006-04-16,GOIÁS,SANTOS,0,0,GO,SP,Empate
2623,2006-04-16,BOTAFOGO-RJ,FORTALEZA,1,0,RJ,CE,Mandante


In [6]:
def get_historical_results(row, cat: str, past_match: int, df: pd.DataFrame):
    
    if cat == 'home':
        variable = 'Clube_1'
    else:
        variable = "Clube_2"
        
    club = row[variable]
    
    match_day = row["Data"].date()
    aux = df.query("{}=='{}' and Data<'{}'".format(variable, club, match_day)).tail(past_match)
    
    return aux["{}_Gols".format(variable)].sum()

In [7]:
cats = ["home", "away"]
pasts = [1, 2, 3]

combinations = itertools.product(cats, pasts)
df_aux = data

for c in combinations:
    categorie = c[0]
    past_matchs = c[1]
    column_name = "{}_gols_ultimos_{}".format(categorie, str(past_matchs))
    
    data[column_name] = data.apply(get_historical_results, axis=1, cat=categorie, past_match=past_matchs, df=df_aux)

In [8]:
#aux = data.melt(id_vars=["Data", "Vencedor_Cat"], value_vars=["Clube_1", "Clube_2"]).replace(["Clube_1", "Clube_2"],
#["Mandante", "Visitante"])

In [9]:
data = data.drop(columns=["Clube_1_Gols", "Clube_2_Gols"])

In [10]:
data = data.drop(columns=["Clube_1", "Clube_2"])

In [11]:
#data = data.query("Vencedor_Cat!='Empate'")

In [12]:
y = data["Vencedor_Cat"]
x = pd.get_dummies(data.drop(columns=["Data", "Vencedor_Cat"]))

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2)

In [14]:
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [15]:
y_pred = clf.predict(x_test)

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      Empate       0.24      0.04      0.06       277
    Mandante       0.49      0.96      0.65       521
   Visitante       0.50      0.01      0.02       266

    accuracy                           0.48      1064
   macro avg       0.41      0.34      0.25      1064
weighted avg       0.43      0.48      0.34      1064



In [17]:
accuracy_score(y_test, y_pred)

0.4830827067669173